In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, BatchNormalization, Activation, Dense, Flatten, Dropout, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics  import binary_crossentropy, categorical_crossentropy, mean_squared_error
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.models import model_from_json
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPUs Availavle: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
ROOT_DIR = os.path.abspath('./')
DATA_PATH = os.path.join('Path")
DATA_PATH

In [ ]:
train_path = os.path.join(DATA_PATH, 'Train')
train_path

In [ ]:
test_path = os.path.join(DATA_PATH, 'Test')
test_path

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.1, rescale=1. / 255)

In [ ]:
train_batches = train_datagen.flow_from_directory(directory=train_path, 
                                            target_size=(234,234), 
                                            classes=['NOR', 'ABNOR'], 
                                            batch_size=64,
                                            subset='training')

In [ ]:
train_batches.class_indices

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), 
            train_batches.classes)

In [ ]:
class_weights

In [ ]:
class_weights = {
    0: 0.69449404,
    1: 1.78538641
}

In [ ]:
validation_batches = train_datagen.flow_from_directory(directory=train_path, 
                                            target_size=(234,234), 
                                            classes=['NOR', 'ABNOR'], 
                                            batch_size=64,
                                            subset='validation')

In [ ]:
validation_batches.class_indices

In [ ]:
test_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(directory=test_path, 
                                                                        target_size=(234,234), 
                                                                        classes=['NOR', 'ABNOR'], 
                                                                        batch_size=64)

In [ ]:
test_batches.class_indices

In [ ]:
len(train_batches)

In [ ]:
# Custom Function to calculate F1 Score 
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
checkpoint_filepath = 'C:\\Users\\anikk\\ECG MOD\\Checkpoints\\best.h5'

In [ ]:
callbacks = ModelCheckpoint(filepath= checkpoint_filepath,  monitor="val_accuracy", verbose=1, save_best_only=True, 
                            save_weights_only=True, mode="max")

# Networks

# Custom Model_1

In [ ]:
model = Sequential()

model.add(Conv2D(12, (3, 3), padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model.add(Conv2D(12, (3, 3), padding='same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(3, 3), strides= 3))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3), padding='same', activation = 'relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(3, 3), strides= 3))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(144, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
model.add(Dense(2, activation='sigmoid'))

# Custom Model_2

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(234, 234, 3)))
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

# VGG 16

In [ ]:
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(234,234,3))

In [ ]:
base_model_vgg16.summary()

In [ ]:
for layer in base_model_vgg16.layers:
    base_model_vgg16.trainable = True

In [ ]:
for i, layer in enumerate(base_model_vgg16.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model = Sequential()

model.add(base_model_vgg16)

model.add(Flatten())

model.add(Dense(2, activation='sigmoid'))

# INCEPTION V3

In [ ]:
base_model_inceptionv3 = InceptionV3(weights=None, include_top=False, input_shape=(234,234,3))

In [ ]:
base_model_inceptionv3.summary()

In [ ]:
for layer in base_model_inceptionv3.layers:
    base_model_inceptionv3.trainable = True

In [ ]:
for i, layer in enumerate(base_model_inceptionv3.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model = Sequential()

model.add(base_model_inceptionv3)

model.add(Flatten())

model.add(Dense(9, activation='softmax'))

# Model training

In [ ]:
model.summary()

In [ ]:
history = model.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=5, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve
plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
#Model Evalution
score, acc, f1_score  = model.evaluate(x=test_batches, steps=len(test_batches))
print('Test score:', score)
print('Test accuracy:', acc)
print('Test F1 Score:', f1_score)

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model
fer_json = model_0.to_json()
with open("Path", "w") as json_file:
    json_file.write(fer_json)
model_0.save_weights("Path")